## Calculate Fst and other popgen statistics

In [13]:
import ipyrad.analysis as ipa
import numpy as np

In [14]:
# init a popgen analysis object
pop = ipa.popgen(
    name="ped", 
    workdir="analysis-popgen", 
    data="./pedicularis/clust85_min12_outfiles/clust85_min12.loci",
)

In [15]:
pop.results

fst     Empty DataFrame
Columns: []
Index: []
pi      Empty DataFrame
Columns: [0]
Index: []
theta   Empty DataFrame
Columns: [0]
Index: []

In [20]:
class Self:
    def __init__(self):
        self.data = np.array([
            [0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0],
            [1, 1, 1, 0, 0], 
            [1, 1, 1, 1, 0],
            [1, 1, 1, 1, 1]
        ])

self = Self()
self.data

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]])

In [51]:
def _fst(self, pop1idx, pop2idx):
    """
    Calculate population fixation index Fst after filtering.
    The variance of allele frequencies between populations.
    Wier and Cockerham (1984) estimator based on average number of 
    pairwise differences: 
       (pi_between - pi_within) / pi_between
    """
    pop1 = self.data[pop1idx, :]
    pop2 = self.data[pop2idx, :]
    popa = self.data[pop1idx + pop2idx, :]
    
    

In [52]:
pop1idx = [0, 1]
pop2idx = [2, 3, 4]
_fst(self, pop1idx, pop2idx)

In [57]:
pop1 = self.data[pop1idx, :]
pop2 = self.data[pop2idx, :]
popa = self.data[pop1idx + pop2idx, :]

pop1, pop2, popa

(array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), array([[1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1]]), array([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1]]))

In [71]:
(np.mean(popa, axis=0) - np.mean(pop2, axis=0)) / (np.mean(popa, axis=0))

array([-0.66666667, -0.66666667, -0.66666667, -0.66666667, -0.66666667])

In [126]:
popa

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0],
       [1, 1, 1, 1, 1]])

In [84]:
np.count_nonzero(pop2, axis=0)

array([3, 3, 3, 2, 1])

In [125]:
a = np.mean(popa, axis=0)
mask = a > 0.5
a[a > 0.5] = abs(a[a > 0.5] - 1)
a

array([0.4, 0.4, 0.4, 0.4, 0.2])